# DACS Elevator Radio Producer

In [1]:
# Install all relied libraries
#!pip install -r requirements.txt

# Initialize
from data.base import store_lyrics, read_cleaned_data
from model.base import get_keywords

c:\Users\P70086556\Anaconda3\envs\nlp_project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Get data from billboard top 100 不要重新运行
If this year's data is stored then it won't grab again. You can directly use them.

In [3]:
store_lyrics(year = None)

Illegal input year. Setting year to be the current.
Searching for "Heat Waves" by Glass Animals...
Done.
Searching for "As It Was" by Harry Styles...
Done.
Searching for "Stay" by The Kid LAROI & Justin Bieber...
Done.
Searching for "Easy On Me" by Adele...
Done.
Searching for "Shivers" by Ed Sheeran...
Done.
Searching for "First Class" by Jack Harlow...
Done.
Searching for "Big Energy" by Latto...
Done.
Searching for "Ghost" by Justin Bieber...
Done.
Searching for "Super Gremlin" by Kodak Black...
Done.
Searching for "Cold Heart (PNAU Remix)" by Elton John & Dua Lipa...
Done.
Searching for "Wait For U" by Future Featuring Drake & Tems...
Done.
Searching for "About Damn Time" by Lizzo...
Done.
Searching for "Bad Habits" by Ed Sheeran...
Done.
Searching for "Thats What I Want" by Lil Nas X...
Done.
Searching for "Enemy" by Imagine Dragons X JID...
Done.
Searching for "Industry Baby" by Lil Nas X & Jack Harlow...
Done.
Searching for "abcdefu" by GAYLE...
Done.
Searching for "Need To Know

## Get data from a playlist from spotify 不要重新运行
### How to find playlist ID:
![playlist id](imgs/playlist_id.png)

Or if you share the playlist with link, find the string before `?si=`:  
https://open.spotify.com/playlist/**37i9dQZF1DX5Ejj0EkURtP**?si=a1e0243dd67c4cc3


In [4]:
store_lyrics(playlist_id="1q2ztYCQCBrq14wFEJKOB8")

Searching for "Apocalypse" by Cigarettes After Sex...
Done.
Searching for "Sunsetz" by Cigarettes After Sex...
Done.
Searching for "Cry" by Cigarettes After Sex...
Done.
Searching for "K." by Cigarettes After Sex...
Done.
Searching for "My Jinji" by Sunset Rollercoaster...
Done.
Searching for "I Know You Know I Love You" by Sunset Rollercoaster...
Done.
Searching for "Jellyfish" by Sunset Rollercoaster...
Done.
Searching for "我是一隻魚 I'm a Fish" by Sunset Rollercoaster...
Done.
Searching for "Blues" by Sunset Rollercoaster...
Done.
Searching for "Welcome To" by Sunset Rollercoaster...
Specified song does not contain lyrics. Rejecting.
There's error when getting lyrcis of 'Welcome To' by Sunset Rollercoaster
'NoneType' object has no attribute 'lyrics'
Searching for "Vanilla" by Sunset Rollercoaster...
Done.
Searching for "Villa" by Sunset Rollercoaster...
Done.
Searching for "I Wanna Be Yours" by Arctic Monkeys...
Done.
Searching for "I Wanna Be Adored - Remastered" by The Stone Roses...


## Pre-process:

In [2]:
address = '2019'
lyrics_tokens = read_cleaned_data(address,stem_words = False)

In [16]:
print(lyrics_tokens["98_See You Again"])

[['it', 'been', 'long', 'day', 'without', 'you', 'my', 'friend'], ['and', 'i', 'will', 'tell', 'you', 'all', 'about', 'it', 'when', 'i', 'see', 'you', 'again'], ['we', 'have', 'come', 'long', 'way', 'from', 'where', 'we', 'began'], ['oh', 'i', 'will', 'tell', 'you', 'all', 'about', 'it', 'when', 'i', 'see', 'you', 'again'], ['when', 'i', 'see', 'you', 'again'], ['dang', 'who', 'knew'], ['all', 'planes', 'we', 'flew'], ['good', 'things', 'we', 'have', 'been', 'through'], ['that', 'i', 'will', 'be', 'standing', 'right', 'here', 'talking', 'to', 'you'], ['bout', 'another', 'path'], ['i', 'know', 'we', 'loved', 'to', 'hit', 'road', 'and', 'laugh'], ['but', 'something', 'told', 'me', 'that', 'it', 'would', 'not', 'last'], ['had', 'to', 'switch', 'up'], ['look', 'at', 'things', 'different', 'see', 'bigger', 'picture'], ['those', 'were', 'days'], ['hard', 'work', 'forever', 'pays'], ['now', 'i', 'see', 'you', 'in', 'better', 'place', 'see', 'you', 'in', 'better', 'place'], ['uh'], ['how', 'ca

## Get topics:

In [3]:
get_keywords(lyrics_tokens)

Song 100_Robbery has topic [[('gucci', 0.2425), ('slur', 0.2287), ('hennessy', 0.2276), ('dad', 0.2106), ('boy', 0.1999)]]
Song 10_Sucker has topic [[('sucker', 0.2862), ('subliminal', 0.27), ('baby', 0.232), ('tattoo', 0.2251), ('road', 0.2133)]]
Song 11_High Hopes has topic [[('hopes', 0.3455), ('destiny', 0.3086), ('prophecy', 0.3024), ('mama', 0.2691), ('rise', 0.2667)]]
Song 12_Thank U Next has topic [[('song', 0.4449), ('songs', 0.3535), ('thanking', 0.3037), ('thankful', 0.2977), ('ricky', 0.2867)]]
Song 13_Truth Hurts has topic [[('bitch', 0.3449), ('man', 0.2692), ('chick', 0.2595), ('boy', 0.2478), ('hair', 0.2321)]]
Song 14_Dancing With A Stranger has topic [[('dancing', 0.2937), ('stranger', 0.2071), ('baby', 0.204), ('tonight', 0.1886), ('control', 0.1743)]]
Song 15_Senorita has topic [[('señorita', 0.4001), ('runnin', 0.2693), ('like', 0.2257), ('kiss', 0.2127), ('tequila', 0.1906)]]
Song 16_I Dont Care has topic [[('baby', 0.3189), ('party', 0.2981), ('dance', 0.2576), (

KeyboardInterrupt: 